In [1]:
# librerie

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql import Row
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import StandardScaler
from pyspark.mllib.clustering import KMeans

In [2]:
# sessione

sc = SparkContext(appName="DDAM_Project", master="local[*]")
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("DDAM_Project") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
sdf = spark.read.csv("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa04/credit_train.csv", sep=",",
                     inferSchema=True, header=True)

columns = sdf.schema.names

# rinominare le colonne sotituendo lo spazio con l'underscore
for col in columns:
    sdf = sdf.withColumnRenamed(col, col.replace(' ', '_'))

columns = sdf.schema.names

sdf.printSchema()

root
 |-- Loan_ID: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Loan_Status: string (nullable = true)
 |-- Current_Loan_Amount: integer (nullable = true)
 |-- Term: string (nullable = true)
 |-- Credit_Score: integer (nullable = true)
 |-- Annual_Income: integer (nullable = true)
 |-- Years_in_current_job: string (nullable = true)
 |-- Home_Ownership: string (nullable = true)
 |-- Purpose: string (nullable = true)
 |-- Monthly_Debt: double (nullable = true)
 |-- Years_of_Credit_History: double (nullable = true)
 |-- Months_since_last_delinquent: string (nullable = true)
 |-- Number_of_Open_Accounts: integer (nullable = true)
 |-- Number_of_Credit_Problems: integer (nullable = true)
 |-- Current_Credit_Balance: integer (nullable = true)
 |-- Maximum_Open_Credit: integer (nullable = true)
 |-- Bankruptcies: string (nullable = true)
 |-- Tax_Liens: string (nullable = true)



### - Funzioni

In [4]:
def get_nbr_nulls(spark_df, view_name, columns, print_result = True):
    """funzione per ottenere il numero di valori nulli presenti in ogni attributo"""
    
    spark_df.createOrReplaceTempView(view_name)
    
    Project = []
    for col in columns:
        Project.append('SUM(CASE WHEN {0} IS NULL THEN 1 ELSE 0 END) AS {0}'.format(col))
    Project = ', '.join(Project)

    sql = """\
    SELECT {0}
    FROM {1}\
    """.format(Project, view_name)
    
    nbr_nulls = spark.sql(sql).first().asDict()
    
    if print_result:
        for key, value in nbr_nulls.items():
            print(key + ':', '{:>10}'.format(value))
        
    return nbr_nulls

In [5]:
nbr_nulls = get_nbr_nulls(spark_df = sdf, view_name = 'Bank_Loan_Dataset', columns = columns)

Loan_ID:        514
Customer_ID:        514
Loan_Status:        514
Current_Loan_Amount:        514
Term:        514
Credit_Score:      19668
Annual_Income:      19668
Years_in_current_job:        514
Home_Ownership:        514
Purpose:        514
Monthly_Debt:        514
Years_of_Credit_History:        514
Months_since_last_delinquent:        514
Number_of_Open_Accounts:        514
Number_of_Credit_Problems:        514
Current_Credit_Balance:        514
Maximum_Open_Credit:        516
Bankruptcies:        514
Tax_Liens:        514


In [6]:
def get_nbr_distincts(spark_df, view_name, columns, print_result = True):
    """funzione per ottenere il numero di valori distinti di ciascun attributo.
    il valore nullo non viene contato come valore distinto"""
    
    spark_df.createOrReplaceTempView(view_name)

    Project = []
    for col in columns:
        Project.append('COUNT(DISTINCT {0}) AS {0}'.format(col))
    Project = ', '.join(Project)

    sql = """\
    SELECT {0}
    FROM {1}\
    """.format(Project, view_name)
    
    nbr_distincts = spark.sql(sql).first().asDict()
    
    if print_result:
        for key, value in nbr_distincts.items():
            print(key + ':', '{:>10}'.format(value))

    return nbr_distincts

In [7]:
nbr_distincts = get_nbr_distincts(spark_df = sdf, view_name = 'Bank_Loan_Dataset', columns = columns)

Loan_ID:      81999
Customer_ID:      81999
Loan_Status:          2
Current_Loan_Amount:      22004
Term:          2
Credit_Score:        324
Annual_Income:      36174
Years_in_current_job:         12
Home_Ownership:          4
Purpose:         16
Monthly_Debt:      65765
Years_of_Credit_History:        506
Months_since_last_delinquent:        117
Number_of_Open_Accounts:         51
Number_of_Credit_Problems:         14
Current_Credit_Balance:      32730
Maximum_Open_Credit:      44596
Bankruptcies:          9
Tax_Liens:         13


# Attributi con Data Type incoerente

controlliamo alcuni "attributi problematici", cioè quelli con un numero di valori distinti basso (inferiore a 200) ma che hanno valori di tipo numerico. Se necessario dobbiamo modificare il loro data type.

In [8]:
# attributi problematici:

problematic_columns = []
for key, value in nbr_distincts.items():
    if value < 200 and key not in ['Loan_Status', 'Term', 'Home_Ownership', 'Purpose']:
        problematic_columns.append(key)
        print(key + ':', '{:>10}'.format(value))

Years_in_current_job:         12
Months_since_last_delinquent:        117
Number_of_Open_Accounts:         51
Number_of_Credit_Problems:         14
Bankruptcies:          9
Tax_Liens:         13


In [9]:
# Check dei valori distinti degli attributi problematici

for col in problematic_columns:
    sdf.select(col).distinct().orderBy(col).show(150)

+--------------------+
|Years_in_current_job|
+--------------------+
|                null|
|              1 year|
|           10+ years|
|             2 years|
|             3 years|
|             4 years|
|             5 years|
|             6 years|
|             7 years|
|             8 years|
|             9 years|
|            < 1 year|
|                 n/a|
+--------------------+

+----------------------------+
|Months_since_last_delinquent|
+----------------------------+
|                        null|
|                           0|
|                           1|
|                          10|
|                         100|
|                         104|
|                         106|
|                         107|
|                         108|
|                          11|
|                         110|
|                         114|
|                         115|
|                         118|
|                          12|
|                         120|
|                  

In [10]:
rdd = sdf.rdd  # di default questa trasformazione genera un RDD di Row()

'''le Row sono tipi particolari di Tuple, quindi sono oggetti immutabili.
in tutto il notebook quindi per modificare gli RDD li trasformiamo temporaneamente (dentro le funzioni)
in RDD di Dictionaries.'''

'le Row sono tipi particolari di Tuple, quindi sono oggetti immutabili.\nin tutto il notebook quindi per modificare gli RDD li trasformiamo temporaneamente (dentro le funzioni)\nin RDD di Dictionaries.'

- 'Years_in_current_job'

i valori sono evidentemente categorici e quindi l'attributo viene mantenuto come stringa.

sono presenti 4222 valori uguali a 'n/a'. Pensiamo si possa trattare di soggetti senza lavoro. Il valore è quindi coerente e viene mantenuto così.

In [11]:
rdd.filter(lambda row: row['Years_in_current_job'] == 'n/a').count()

4222

- 'Months_since_last_delinquent'

consideriamo questo attributo come categorico perché 117 valori distinti non sono abbastanza a nostro avviso per definire l'attributo come numerico (non avrebbe senso ad esempio dividere i valori in Bins) e anche perché la maggior parte dei valori sono uguali a 'NA'.

sono presenti infatti 53141 valori uguali a NA. la nostra interpretazione di questo valore è che il soggetto in questione non ha mai commesso nessun reato (anche perché i valori 'NA' sono la maggior parte). trasformiamo i valori 'NA' in '-1', cioè manteniamo l'attributo come stringa ma lo prepariamo nel caso si volesse trasformare in Intero per analizzare i suoi valori numerici.

In [12]:
rdd.filter(lambda row: row['Months_since_last_delinquent'] == 'NA').count()

53141

In [13]:
def change_value(row):
    d = row.asDict()
    if d['Months_since_last_delinquent'] == 'NA':
        d['Months_since_last_delinquent'] = '-1'
    return Row(**d)

rdd = rdd.map(change_value)

rdd.filter(lambda d: d['Months_since_last_delinquent'] == '-1').count()

53141

- 'Number_of_Open_Accounts'

con 51 valori numerici distinti l'attributo viene considerato categorico. quindi rendiamo stringhe i suoi valori.

In [14]:
def change_value(row):
    d = row.asDict()
    d['Number_of_Open_Accounts'] = str(d['Number_of_Open_Accounts'])
    return Row(**d)

rdd = rdd.map(change_value)

- 'Number_of_Credit_Problems'

con 14 valori numerici distinti l'attributo viene considerato categorico. quindi rendiamo stringhe i suoi valori.

In [15]:
def change_value(row):
    d = row.asDict()
    d['Number_of_Credit_Problems'] = str(d['Number_of_Credit_Problems'])
    return Row(**d)

rdd = rdd.map(change_value)

- 'Bankruptcies'

con 8 valori numerici distinti l'attributo viene mantenuto categorico.

sono presenti 204 valori dell'attributo 'Bankruptcies' uguali a NA. Poichè il valore 0 è presente si pensa possa trattarsi di missing values, quindi li trasformiamo in None.

In [16]:
rdd.filter(lambda d: d['Bankruptcies'] == 'NA').count() + rdd.filter(lambda d: d['Bankruptcies'] is None).count()

718

In [17]:
def change_value(row):
    d = row.asDict()
    if d['Bankruptcies'] == 'NA':
        d['Bankruptcies'] = None
    return Row(**d)

rdd = rdd.map(change_value)

rdd.filter(lambda d: d['Bankruptcies'] is None).count()

718

- 'Tax_Liens'

con 12 valori numerici distinti l'attributo viene mantenuto categorico.

sono presenti 10 valori dell'attributo 'Tax_Liens' uguali a NA. Poichè il valore 0 è presente si pensa possa trattarsi di missing values, quindi li trasformiamo in None.

In [18]:
rdd.filter(lambda d: d['Tax_Liens'] == 'NA').count() + rdd.filter(lambda d: d['Tax_Liens'] is None).count()

524

In [19]:
def change_value(row):
    d = row.asDict()
    if d['Tax_Liens'] == 'NA':
        d['Tax_Liens'] = None
    return Row(**d)

rdd = rdd.map(change_value)

rdd.filter(lambda d: d['Tax_Liens'] is None).count()

524

# gestione degli errori semantici

alcuni attributi hanno dei valori non coerenti con il significato che noi reputiamo possa avere l'attributo (essando sprovvisti di adeguata documentazione su di essi). Alcuni di questi errori sono stati scoperti in fasi più avanzate del progetto (ex Data Understanding).

- il valore dell'attributo 'Current_Credit_Balance' non può essere superiore al valore dell'attributo 'Maximum_Open_Credit'

In [20]:
def semantic_errors(row):
    if row['Current_Credit_Balance'] is not None and row['Maximum_Open_Credit'] is not None:
        return row['Current_Credit_Balance'] < row['Maximum_Open_Credit']
    else:
        return row
    
rdd = rdd.filter(semantic_errors)
rdd.count()

99819

- ci sono 4526 valori dell'attributo 'Credit_Score' con valore superiore a 4000, che è un valore troppo distante rispetto a quelli generici che assume questo attributo (si aggirano intorno a 200).

In [21]:
def semantic_errors(row):
    if row['Credit_Score'] is not None:
        return row['Credit_Score'] < 4000
    else:
        return row
    
rdd = rdd.filter(semantic_errors)
rdd.count()

95293

- ci sono diversi valori dell'attributo 'Maximum_Open_Credit' che sono pari a 99.999.999.  un numero eccessivamente più alto rispetto a tutti gli altri valori e che per questo viene eliminato.

In [22]:
def semantic_errors(row):
    if row['Current_Loan_Amount'] is not None:
        return row['Current_Loan_Amount'] != 99999999
    else:
        return row
    
rdd = rdd.filter(semantic_errors)
rdd.count()

83904

- eliminiamo le righe dove entrambi gli attributi 'Loan_ID' e 'Customer_ID' sono nulli, che corrispondono alle righe dove tutti i valori di tutti gli attributi sono nulli

In [23]:
rdd = rdd.filter(lambda row: not ( (row['Loan_ID'] is None) and (row['Customer_ID'] is None) ))
rdd.count()

83390

- eliminiamo le righe duplicate

In [24]:
rdd = rdd.distinct()
rdd.count()

73230

# mximum open credit di 100 000 000
# annual income

- Problema: ci sono coppie di righe con tutti i valori duplicati eccetto per le due colonne 'Credit_Score' e 'Annual_Income', per le quali uno dei due valori è presente e l'altro è nullo.
- Soluzione: si raggruppa per tutti gli attributi tranne quei due e poi si calcola la media di quei due. In questo modo se le uniche due righe uguali sono quelle con un valore nullo e uno non nullo per quegli attributi, lo media sarà uguale al valore non nullo; se invece ci fossero altre righe ugauli ma con altri valori diversi non nulli per quegli attributi, viene effettivamente calcolata la media, il che è auspicabile considerando che tutto il resto della riga è uguale e quindi si tratta molto probabilmente dello stesso oggetto, duplicato per errore, di cui dunque prendiamo un valore medio tra quelli presenti.

In [25]:
sdf = rdd.toDF()

sdf.createOrReplaceTempView('Bank_Loan_Dataset')

columns_temp = [col for col in columns if col != 'Credit_Score' and col != 'Annual_Income']
Project = ', '.join(columns_temp)

sql = """
SELECT {0}, AVG(Credit_Score) AS Credit_Score, AVG(Annual_Income) AS Annual_Income
FROM Bank_Loan_Dataset
GROUP BY {0}
""".format(Project)

sdf = spark.sql(sql)

del columns_temp

- come si nota i Customer e Loan ID che si ripetevano nel dataset originale erano solo righe duplicate. il dataset pulito non presenta nessuna riga uguale negli ID. possiamo quindi eliminarli. Anche escludendo questi attributi tutte le righe rimangono distinte.

In [26]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT COUNT(*) AS nbr_rows, COUNT(DISTINCT Customer_ID) AS nbr_customers, COUNT(DISTINCT Loan_ID) AS nbr_loans
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

+--------+-------------+---------+
|nbr_rows|nbr_customers|nbr_loans|
+--------+-------------+---------+
|   69057|        69057|    69057|
+--------+-------------+---------+



In [27]:
columns = [col for col in columns if col != 'Customer_ID' and col != 'Loan_ID']

sdf = sdf.select(columns)

# Gestione dei Missing Values

In [28]:
nbr_nulls = get_nbr_nulls(spark_df=sdf, view_name = 'Bank_Loan_Dataset', columns = columns)

Loan_Status:          0
Current_Loan_Amount:          0
Term:          0
Credit_Score:      14838
Annual_Income:      14838
Years_in_current_job:          0
Home_Ownership:          0
Purpose:          0
Monthly_Debt:          0
Years_of_Credit_History:          0
Months_since_last_delinquent:          0
Number_of_Open_Accounts:          0
Number_of_Credit_Problems:          0
Current_Credit_Balance:          0
Maximum_Open_Credit:          1
Bankruptcies:        138
Tax_Liens:          7


Filliamo i Missing values degli attributi "Maximum_Open_Credit", "Bankruptcies" e "Tax_Liens" usando la Media di diversi tipi di raggruppamenti.

per farlo usiamo la sintassi dell'SQL analitico, creando nuove apposite colonne.

In [29]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT *,
    AVG(Maximum_Open_Credit) OVER(PARTITION BY Years_in_current_job,
                                                Home_Ownership,
                                                Number_of_Open_Accounts,
                                                Years_of_Credit_History) AS toFill_Maximum_Open_Credit,
    AVG(Bankruptcies) OVER(PARTITION BY Months_since_last_delinquent,
                                        Number_of_Credit_Problems) AS toFill_Bankruptcies,
    AVG(Tax_Liens) OVER(PARTITION BY Months_since_last_delinquent,
                                    Number_of_Credit_Problems) AS toFill_Tax_Liens
FROM Bank_Loan_Dataset
"""

sdf = spark.sql(sql)

In [30]:
def fill_nulls(row):
    d = row.asDict()
    if d['Maximum_Open_Credit'] is None:
        d['Maximum_Open_Credit'] = int(d['toFill_Maximum_Open_Credit'])
    if d['Bankruptcies'] is None:
        d['Bankruptcies'] = str(d['toFill_Bankruptcies'])
    if d['Tax_Liens'] is None:
        d['Tax_Liens'] = str(d['toFill_Tax_Liens'])
    return Row(**d)

sdf = sdf.rdd.map(fill_nulls).toDF().select(columns)

nbr_nulls = get_nbr_nulls(spark_df=sdf, view_name = 'Bank_Loan_Dataset', columns = columns)

Loan_Status:          0
Current_Loan_Amount:          0
Term:          0
Credit_Score:      14838
Annual_Income:      14838
Years_in_current_job:          0
Home_Ownership:          0
Purpose:          0
Monthly_Debt:          0
Years_of_Credit_History:          0
Months_since_last_delinquent:          0
Number_of_Open_Accounts:          0
Number_of_Credit_Problems:          0
Current_Credit_Balance:          0
Maximum_Open_Credit:          0
Bankruptcies:          0
Tax_Liens:          0


Filliamo i Missing Values degli attributi "Credit_Score" e "Annual_Income" dividendo il dataset in clusters e usando la Media dei valori nei clusters.

Per effettuare il clustering dobbiamo considerare solo le colonne numeriche diverse da 'Credit_Score' e 'Annual_Income'.

In [31]:
columns_categorical = [col.name for col in sdf.schema.fields if isinstance(col.dataType, StringType)]

columns_numerical = [col for col in columns if col not in columns_categorical]

columns_clustering = [col for col in columns_numerical if col != 'Credit_Score' and col != 'Annual_Income']

columns_clustering

['Current_Loan_Amount',
 'Monthly_Debt',
 'Years_of_Credit_History',
 'Current_Credit_Balance',
 'Maximum_Open_Credit']

In [32]:
X = sdf.select(columns_clustering).rdd.map(lambda row: Vectors.dense(row))

X_scaled = StandardScaler(withMean=True, withStd=True).fit(X).transform(X)

# Migliora modello

In [33]:
model = KMeans.train(X_scaled, k=10, maxIterations=100, initializationMode="random")

In [34]:
def append_label(element):
    d = element[0].asDict()
    d['cluster_label'] = str(element[1])
    return Row(**d)

sdf = sdf.rdd.zip(model.predict(X_scaled)).map(append_label).toDF()

In [35]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT *,
    AVG(Credit_Score) OVER(PARTITION BY cluster_label) AS toFill_Credit_Score,
    AVG(Annual_Income) OVER(PARTITION BY cluster_label) AS toFill_Annual_Income
FROM Bank_Loan_Dataset
"""

sdf = spark.sql(sql)

In [36]:
def fill_nulls(row):
    d = row.asDict()
    if d['Credit_Score'] is None:
        d['Credit_Score'] = float(d['toFill_Credit_Score'])
    if d['Annual_Income'] is None:
        d['Annual_Income'] = float(d['toFill_Annual_Income'])
    return Row(**d)

sdf = sdf.rdd.map(fill_nulls).toDF().select(columns)

nbr_nulls = get_nbr_nulls(spark_df=sdf, view_name = 'Bank_Loan_Dataset', columns = columns)

Loan_Status:          0
Current_Loan_Amount:          0
Term:          0
Credit_Score:          0
Annual_Income:          0
Years_in_current_job:          0
Home_Ownership:          0
Purpose:          0
Monthly_Debt:          0
Years_of_Credit_History:          0
Months_since_last_delinquent:          0
Number_of_Open_Accounts:          0
Number_of_Credit_Problems:          0
Current_Credit_Balance:          0
Maximum_Open_Credit:          0
Bankruptcies:          0
Tax_Liens:          0


# *** sei qui

### Label Encoding of Categorical Attributes

creiamo una nuova struttura dati tabulare dove mappiamo ciascun valore distinto degli attributi caegorici con un numero. Questa sarà utile per sostituire òe stringhe degli attributi categorici 

In [46]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT COUNT(*) AS nbr_rows
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

+--------+
|nbr_rows|
+--------+
|   69057|
+--------+



In [47]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT Loan_Status, COUNT(*) AS nbr_rows
FROM Bank_Loan_Dataset
GROUP BY Loan_Status """

spark.sql(sql).show()

+-----------+--------+
|Loan_Status|nbr_rows|
+-----------+--------+
| Fully Paid|   51088|
|Charged Off|   17969|
+-----------+--------+



In [48]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT Loan_Status, AVG(Credit_Score) AS avg_Credit_Score
FROM Bank_Loan_Dataset
GROUP BY Loan_Status """

spark.sql(sql).show()

+-----------+-----------------+
|Loan_Status| avg_Credit_Score|
+-----------+-----------------+
| Fully Paid|719.9645575699363|
|Charged Off|710.2000954350247|
+-----------+-----------------+



In [49]:
sdf.createOrReplaceTempView('Bank_Loan_Dataset')

sql = """
SELECT Loan_Status, AVG(Annual_Income) AS avg_Credit_Score
FROM Bank_Loan_Dataset
GROUP BY Loan_Status """

spark.sql(sql).show()

+-----------+------------------+
|Loan_Status|  avg_Credit_Score|
+-----------+------------------+
| Fully Paid|1410691.1453956058|
|Charged Off|1253940.9389215843|
+-----------+------------------+



In [43]:
# sdf.write.parquet("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa04/bank_loan_status")

In [49]:
# sdf = spark.read.parquet("hdfs://kddrtserver11.isti.cnr.it:9000/user/hpsa04/bank_loan_status")

# Ricordati!

anche se non esistono valori distinti  COUNT(\*)  può differire da COUNT(DISTINCT \*).

perché il primo conta tutte le righe mentre il secondo conta tutte e sole le righe dove non è presente neanche un NULL value

In [ ]:
sql = """
SELECT COUNT(*) AS nbr_rows
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

In [ ]:
sql = """
SELECT COUNT(Credit_Score) AS nbr_rows
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

In [ ]:
sql = """
SELECT COUNT(DISTINCT *) AS nbr_rows
FROM Bank_Loan_Dataset
"""

spark.sql(sql).show()

# tentativi

In [42]:
'''  tentativo non riuscito per fillare i missing values
rdd = sdf.rdd.map(lambda row: row.asDict())

rdd.keyBy(lambda row: (row['Years_in_current_job'],
                             row['Home_Ownership'],
                             row['Number_of_Open_Accounts'],
                             row['Years_of_Credit_History'])).groupByKey()
                             
def fill_null(d):
    if d['Maximum_Open_Credit'] == None:
        d['Maximum_Open_Credit'] = 
    return d

rdd.mapValues(fill_null)
'''

"  tentativo non riuscito\nrdd = sdf.rdd.map(lambda row: row.asDict())\n\nrdd.keyBy(lambda row: (row['Years_in_current_job'],\n                             row['Home_Ownership'],\n                             row['Number_of_Open_Accounts'],\n                             row['Years_of_Credit_History'])).groupByKey()\n                             \ndef fill_null(d):\n    if d['Maximum_Open_Credit'] == None:\n        d['Maximum_Open_Credit'] = \n    return d\n\nrdd.mapValues(fill_null)\n"